In [9]:
from dotenv import load_dotenv
import os

load_dotenv('/Users/p.miroshin/Documents/Masters/2d_year/keys.env')

ACCESS_SECRET_KEY = os.getenv("ACCESS_SECRET_KEY")
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")

In [10]:
import boto3

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    region_name='ru-central1',
    aws_access_key_id=os.environ.get('ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('ACCESS_SECRET_KEY')
)

BUCKET_NAME = 'bucket-first'

def upload_empty_csv(file_name, file_key):
    s3.put_object(Bucket=BUCKET_NAME, Key=file_key, Body=file_name)
    print(f"{file_key} uploaded to bucket {BUCKET_NAME}.")

In [11]:
users_csv = "chat_id,shikimori_username,last_updated\n"
anime_csv = "anime_id,title,average_score,genres,type,episode_duration,episode_count,poster_url\n"
user_anime_csv = "user_anime_id,chat_id,anime_id,status,user_score\n"
recommendations_csv = "recommendation_id,user_id,anime_id,date_recommended,user_score\n"
logs_csv = "log_id,user_id,anime_id,action,details,timestamp\n"

In [10]:
upload_empty_csv(users_csv, 'users.csv')
upload_empty_csv(anime_csv, 'anime.csv')
upload_empty_csv(user_anime_csv, 'user_anime.csv')
upload_empty_csv(recommendations_csv, 'recommendations.csv')
upload_empty_csv(logs_csv, 'logs.csv')

users.csv uploaded to bucket bucket-first.
anime.csv uploaded to bucket bucket-first.
user_anime.csv uploaded to bucket bucket-first.
recommendations.csv uploaded to bucket bucket-first.
logs.csv uploaded to bucket bucket-first.


In [15]:
upload_empty_csv(users_csv, 'users.csv')

users.csv uploaded to bucket bucket-first.


In [1]:
from dotenv import load_dotenv
import os
import boto3
import csv
import io
from datetime import datetime

# Загружаем переменные окружения
load_dotenv('/Users/p.miroshin/Documents/Masters/2d_year/keys.env')
ACCESS_SECRET_KEY = os.getenv("ACCESS_SECRET_KEY")
ACCESS_KEY_ID = os.getenv("ACCESS_KEY_ID")

# Создаем S3 клиент
session = boto3.session.Session()
s3 = session.client(
    service_name="s3",
    endpoint_url="https://storage.yandexcloud.net",
    region_name="ru-central1",
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=ACCESS_SECRET_KEY,
)

BUCKET_NAME = "bucket-first"

def upload_to_s3(file_key, content):
    """
    Загружает содержимое в S3.
    """
    s3.put_object(Bucket=BUCKET_NAME, Key=file_key, Body=content)
    print(f"{file_key} успешно загружен в бакет {BUCKET_NAME}.")

def generate_csv_content(data, fieldnames):
    """
    Генерирует CSV-файл безопасно.
    :param data: Данные для записи (список словарей)
    :param fieldnames: Поля заголовков
    :return: Содержимое файла CSV в формате строки
    """
    output = io.StringIO()
    writer = csv.DictWriter(output, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)
    return output.getvalue()

def initialize_files_with_real_data():
    """
    Создает и загружает CSV-файлы с реальными данными.
    """
    # Таблица списков рекомендаций
    recommended_lists = [
        {
            "list_id": "1",
            "name": "Легендарные аниме",
            "description": "Культовые аниме всех времён",
            "type": "Общий",
            "genre": "Action",
            "year": None,
            "created_by": "505096036",
            "created_at": datetime.utcnow().isoformat(),
            "updated_at": datetime.utcnow().isoformat(),
        },
        {
            "list_id": "2",
            "name": "Аниме для новичков",
            "description": "Идеальные тайтлы для начала знакомства с аниме",
            "type": "Для новичков",
            "genre": None,
            "year": None,
            "created_by": "505096036",
            "created_at": datetime.utcnow().isoformat(),
            "updated_at": datetime.utcnow().isoformat(),
        },
        {
            "list_id": "3",
            "name": "Лучшие комедии",
            "description": "Весёлые комедийные аниме",
            "type": "Комедия",
            "genre": "Comedy",
            "year": "2023",
            "created_by": "123456789",
            "created_at": datetime.utcnow().isoformat(),
            "updated_at": datetime.utcnow().isoformat(),
        },
    ]
    recommended_lists_content = generate_csv_content(
        recommended_lists,
        fieldnames=[
            "list_id",
            "name",
            "description",
            "type",
            "genre",
            "year",
            "created_by",
            "created_at",
            "updated_at",
        ],
    )

    # Таблица привязанных аниме к спискам
    recommended_list_items = [
        {"list_id": "1", "anime_id": "101"},
        {"list_id": "1", "anime_id": "102"},
        {"list_id": "1", "anime_id": "103"},
        {"list_id": "2", "anime_id": "201"},
        {"list_id": "2", "anime_id": "202"},
        {"list_id": "2", "anime_id": "203"},
        {"list_id": "3", "anime_id": "301"},
        {"list_id": "3", "anime_id": "302"},
        {"list_id": "3", "anime_id": "303"},
    ]
    recommended_list_items_content = generate_csv_content(
        recommended_list_items, fieldnames=["list_id", "anime_id"]
    )

    # Проверка таблиц
    debug_csv(recommended_lists_content, expected_columns=9)
    debug_csv(recommended_list_items_content, expected_columns=2)

    # Загрузка CSV в S3
    upload_to_s3("recommended_lists.csv", recommended_lists_content)
    upload_to_s3("recommended_list_items.csv", recommended_list_items_content)

def debug_csv(content, expected_columns):
    """
    Проверка корректности содержимого CSV перед загрузкой.
    """
    rows = content.strip().split("\n")
    for i, row in enumerate(rows):
        columns = row.split(",")
        if len(columns) != expected_columns:
            print(f"Проблемная строка {i + 1}: {row}")
        else:
            print(f"Строка {i + 1} корректна.")

# Создаем и проверяем данные
initialize_files_with_real_data()

Строка 1 корректна.
Строка 2 корректна.
Строка 3 корректна.
Строка 4 корректна.
Строка 1 корректна.
Строка 2 корректна.
Строка 3 корректна.
Строка 4 корректна.
Строка 5 корректна.
Строка 6 корректна.
Строка 7 корректна.
Строка 8 корректна.
Строка 9 корректна.
Строка 10 корректна.
recommended_lists.csv успешно загружен в бакет bucket-first.
recommended_list_items.csv успешно загружен в бакет bucket-first.
